**Compute and add similarity features to pairs**

In [82]:
%matplotlib inline
import json
import pickle
import string
import re
import nltk
import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pp = pprint.PrettyPrinter(width=100)

import warnings
warnings.filterwarnings('ignore')

In [68]:
pairs_authorf = pickle.load(open('/home/username/workspace/data/output/_jobs/features/pairs_post_author_features.pickle','rb'))
# pairs_authorf2 = pickle.load(open('/home/username/workspace/data/output/_jobs/features/pairs_post_author_features_pearson.pickle','rb'))

In [74]:
author_features.keys()

dict_keys(['subreddit_category_tfidf_pca', 'entity_pca5', 'subreddit_topcategory_tfidf_pca', 'subreddit_category_raw_pca', 'subreddit_topcategory_raw_pca', 'subreddit_pca5'])

In [69]:
subreddit_category_raw_pca = pickle.load(open('/home/username/workspace/data/output/_jobs/features/subreddit_category_raw_pca.pickle', 'rb'))
subreddit_category_tfidf_pca = pickle.load(open('/home/username/workspace/data/output/_jobs/features/subreddit_category_tfidf_pca.pickle', 'rb'))

subreddit_topcategory_raw_pca = pickle.load(open('/home/username/workspace/data/output/_jobs/features/subreddit_topcategory_raw_pca.pickle', 'rb'))
subreddit_topcategory_tfidf_pca = pickle.load(open('/home/username/workspace/data/output/_jobs/features/subreddit_topcategory_tfidf_pca.pickle', 'rb'))

author_features = {}
author_features['subreddit_category_raw_pca'] = subreddit_category_raw_pca
author_features['subreddit_category_tfidf_pca'] = subreddit_category_tfidf_pca

author_features['subreddit_topcategory_raw_pca'] = subreddit_topcategory_raw_pca
author_features['subreddit_topcategory_tfidf_pca'] = subreddit_topcategory_tfidf_pca


In [70]:
author_category_entity_vec = pickle.load(open('/home/username/workspace/data/output/_jobs/features/author_category_entity_vec.pickle','rb'))
author_topcategory_entity_vec = pickle.load(open('/home/username/workspace/data/output/_jobs/features/author_topcategory_entity_vec.pickle','rb'))

In [73]:
author_subreddit_pca5 = pickle.load(open('/home/username/workspace/data/output/_jobs/features/author_subreddit_pca5.pickle','rb'))
author_features['subreddit_pca5'] = author_subreddit_pca5

author_entity_pca5 = pickle.load(open('/home/username/workspace/data/output/_jobs/features/author_entity_pca5.pickle','rb'))
author_features['entity_pca5'] = author_entity_pca5

In [71]:
all_cats = []
for a,cats in author_category_entity_vec.items():
    for c, vecs in cats.items():
        if c not in all_cats:
            all_cats.append(c)
all_cats = sorted(all_cats, key=lambda x: x)
print(len(all_cats))

all_topcats = []
for a,cats in author_topcategory_entity_vec.items():
    for c, vecs in cats.items():
        if c not in all_topcats:
            all_topcats.append(c)
all_topcats = sorted(all_topcats, key=lambda x: x)
print(len(all_topcats))

745
21


In [78]:
def get_full_dict(tmp_cats, all_categories):
    cats = {}
    for k in all_categories:
        cats[k] = tmp_cats[k] if k in tmp_cats else 0

    return cats

def common_entries(*dcts):
    for i in set(dcts[0]).intersection(*dcts[1:]):
        yield (i,) + tuple(d[i] for d in dcts)

In [80]:
import scipy
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances


def get_similarity(op, a, _dict, typ):
    opv = _dict[op][typ]
    av = _dict[a][typ]
    return cosine_similarity(opv,av)[0][0]
#     return sim(opv,av)

def get_similarity2(op, a, _dict, typ, all_categories):
    opv = _dict[op]
    av = _dict[a]
    common = common_entries(opv, av)
    tmp_cats1 = {}
    for c in common:
        if typ is not None:
            cos_sim = cosine_similarity(c[1][typ],c[2][typ])
        else:
            cos_sim = cosine_similarity(c[1].reshape(1, -1),c[2].reshape(1, -1))
        tmp_cats1[c[0]] = cos_sim[0][0]
    return get_full_dict(tmp_cats1, all_categories)

def get_subreddit_cats_pca_similarities(x):
    op = x['op']['author']
    a = x['a']['author']

    x['features']['author']['subreddit_category_raw_pca'] = get_similarity2(op,a,author_features['subreddit_category_raw_pca'],None,all_cats)
    x['features']['author']['subreddit_category_tfidf_pca'] = get_similarity2(op,a,author_features['subreddit_category_tfidf_pca'],None,all_cats)

    x['features']['author']['subreddit_topcategory_raw_pca'] = get_similarity2(op,a,author_features['subreddit_topcategory_raw_pca'],None,all_topcats)
    x['features']['author']['subreddit_topcategory_tfidf_pca'] = get_similarity2(op,a,author_features['subreddit_topcategory_tfidf_pca'],None,all_topcats)

    return x

def get_subreddit_pca5_features(x):
    op = x['op']['author']
    a = x['a']['author']
#     author_features['subreddit_pca5']
    x['features']['author']['subreddit_pca5'] = get_similarity(op,a,author_features['subreddit_pca5'],None)
#     x['features']['author']['subreddit_pca5_pearson'] = get_pearson_sim(op,a,author_features['subreddit_pca5'],None)
    return x

def get_entity_pca5_features(x):
    op = x['op']['author']
    a = x['a']['author']
    x['features']['author']['entity_pca5'] = get_similarity(op,a,author_features['entity_pca5'],None)
#     x['features']['author']['entity_pca5'] = get_pearson_sim(op,a,author_features['entity_pca5'],None)
    return x

# pairs_authorf = list(map(lambda x: get_subreddit_pca5_features(x), pairs_authorf))
# pairs_authorf = list(map(lambda x: get_entity_pca5_features(x), pairs_authorf))

pairs_authorf = list(map(lambda x: get_subreddit_cats_pca_similarities(x), pairs_authorf))

pairs_authorf[0]

{'a': {'author': 'DaystarEld',
  'comments': [{'archived': False,
    'author': 'DaystarEld',
    'author_flair_css_class': None,
    'author_flair_text': '11Δ',
    'body': 'The majority of your points seem predicated on the idea that a) Basic Income requires a flat tax or rejects progressive taxation, and b) that absolutely every social program must be scrapped for a BI to take effect.\n\nNeither of these are true. Of course there are many conservatives or right-libertarian who would prefer those outcomes, but there are also many liberals and left-libertarians who do not.\n\nThe title of your CMV is worded incorrectly. You are arguing against not Basic Income, but the questions of how to pay for it and what exactly would be "replaced."\n\nIf that doesn\'t change your view, I\'m really not sure what would.  The post is very well written and laid out, it\'s just a perfectly shot arrow aimed at the wrong target.',
    'controversiality': 0,
    'created_utc': '1427651011',
    'delta': 

In [81]:
pickle.dump(pairs_authorf, open('/home/username/workspace/data/output/_jobs/features/pairs_post_author_features.pickle','wb'))